---
layout: post
title:  "A Natural Language Processing project recipe"
date:   2023-04-07 10:14:54 +0700
categories: MachineLearning
---

# Introduction

Natural Language Processing (NLP) is a branch of machine learning that analyze to understand and able to generate human language, aiding decision making. NLP has a wide range of applications, including chatbots, sentiment analysis, machine translation and speech recognition. 

- Sentiment analysis invovles analyzing the emotional aspect of a piece of text. It is to determine whether it is positive, negative or neutral (whether people like or dislike the product)

- Named Entity Recognition: this involves identifying and classifying named entities in a sentence. For example, a particular word can be a person, organization, date, or location.

- Part of speech tagging: this is to tag each word in a sentence with its part of speech. We need this to understand the meaning of the whole sentence.

- Text classification has tasks such as classifying an email as spam or not.

- Machine translation: assisting language translation with suggestion automatically.

- Chatbot: converse with users on narrow topics


# Text preprocessing

Some usual preprocessing such as lowercase all the words, remove punctuations.

## Stemming
Stemming is when we truncate the words to their stem. For example: going and go and gone all have the stem 'go'.

## Tokenization
In general, the prepocessing of an NLP task involves breaking down text into individual tokens. Tokens are root syllables that make up that word. The process of breaking down text into tokens is called tokenization. After tokenization, we convert tokens into numbers, also called vectorized. Since a word can be represented as a vector of multiple values - each value is a dimension of that word in the latent space.

Sometimes the words co-occur in n-grams. For example 'the United States' can be treated as an n-gram of length three.

## One hot encoding

We collect a corpus of unique words from document (mixing them altogether). Then we start to encode the interested sentence into a sparse matrix of 0 and 1, with each row to be one word in the corpus, and each column to be one word in the sentence. In each row, we use number 1 to represent the appearance of the word in the sentence, other words to be 0. For example:

The fox jumps over the cat.

The: 1 0 0 0 1 0

fox: 0 1 0 0 0 0

jump: 0 0 1 0 0 0

..

Value of 1 indicate the presence of a word (row) at a particular position (column) in the sentence. 

## Word vector

While one hot encoding captures word location only, word vector (word embedding) captures both meaning and location. With a reprensented vector in the latent space, NLP models can also learn linguistic features automatically. Since these embeddings capture semantic and syntactic relationships between words they can be used as input features for natural language processing (NLP) models.

The overarching concept is a word space (latent space) that assigns meaning to each location. Initially, each word has a random location within the space. Then we consider the frequency in which two words show up together in a big corpus, that would adjust those word locations gradually, hence the words can pick up and shift their meaning. For example, for each word, we consider three words before and three words after to be its context. Apart from similarity in meaning, we can also do simple algebra on this space (King - Man + Woman = Queen), this means that word representation space is a multi cluster distribution. In other words, we don't just find similar words together, on a global scale, the space also forms clusters of words as its inherent structure. Note that the inherent structure can display bias learnt from the natural language, for example, man is to coder as woman is to homemaker.

### word2vec

In the paper "Efficient Estimation of Word Representations in Vector Space", 2013, the authors use a proposed techniques for measuring the quality of the resulting word vector, with the assumption that not only will similar words tend to be close, but words can have multiple degrees of similarity. For example, nouns can have multiple endings, and if we search for similar words in a subspace of the original vector space, we can find words with similar endings. And the similarity goes beyond simple syntactic regularities. We can perform simple algebraic operations on the vector space as well. For example: King - Man + Woman = Queen. This is surprising since the inherent structure of the vector space allows this, and words learn their position in the space after computation about them and their context are carried on.

The authors want to maximize this kind of accuracy in vector operations and optimize the computation. Apart from the usual feed forward and the recurrent neural net, they  propose a new log linear neural network architectures for Word2Vec. We would look into those models respectively. A probabilistic feedforward neural net for language model has input, project, hidden and output layers. At input, N previous words are encoded using 1-of-V coding, with V being the size of the vocabulary. The input layer is then projected onto a projection layer P that has dimensionality N x D, using a shared project matrix. The architecture becomes complex between projection and hidden layer. The output layer is all the words in the vocabulary, since it has V dimensionality. Hence the complexity mostly resides in hidden and output layers. To reduce this, the authors use hierarchical softmax and vocabulary is represented as a Huffman binary tree (with frequent words assigned short binary codes) reducing the output.

A recurrent neural net theoretically can represent more complex data patterns than a shallow net.  It doesn't have projection layer, and it has recurrent matrix that connects hidden layers. This forms some sort of short term memory since it stores and update the hidden layer in each time step. Using hierarchical softmax can reduce some complexity in hidden to output layer.

For their contribution, they propose two new architectures that train model in two steps: first, learn the word vectors, then train n-gram model on top of those. They also train parallelly on CPU of different machines to investigate different models. The framework is named DistBelief, with mini batch gradient descent and adaptive learning rate of Adagrad.

The first architecture (a log linear one) is similar to the feedforward, where they remove the nonlinear hidden layer, and average the word vectors (projected them into the same position). This is called a bag of word (BOW) model since they don't take into account the order of words. During calculation, they count 4 words in the past and 4 words in the future of the current word. Continuous BOW predicts the word based on those context words.

The Continuous Skip-Gram model predicts context words given a target word. Specifically, the current word is inputted into a log linear classifier with continuous project layer and predict the word before and after it.

<img width="718" alt="Screen Shot 2023-04-07 at 16 54 47" src="https://user-images.githubusercontent.com/7457301/230588558-3a9399e5-4f31-4c43-8966-54da221deb8f.png">

They train on Google News corpus (6B tokens) and vocabulary size is restricted at 1 million most frequent words at 640 dimension (a vector of 640 to represent each word). To avoid curse of dimensionality (adding dimension or data diminishes the performance at some point), they increase both dimensions and data points at the same time.

The after trained space can do simple algebra. For example, X = vector('biggest') - vector('big') + vector('small'). If we search in the vector space for words that cosine similar (Euclidean distance) to X, we might find 'smallest'. This kind of training on high dimensional word vectors can answer other semantic relationships too. For example, France to Paris is as Germany to Berlin. A word space with such inherent relationship can be very useful for other down the line machine learning tasks.

Their architectures perform well, better than the RNN and feedforward network, even in the Microsoft Sentence Completion Challenge.

### GloVe

GloVe (Global Vectors for Word Representation) is a word embedding method that came out in 2014. It is designed to learn word representations that capture the meaning of words in a more global context. It is a new global log bilinear regression model that combines the advantages of the two major model families: global matrix factorization and local context window methods. It results in a vector space with meaningful substructure, scoring high on word analogy task and named entity recognition task.

How did they do that? They argue that global log bilinear regression can produce linear direction of meaning (the simple algebra part). And then they propose a weighted least squares to train on global word-word co-occurence counts, this is to make use of global statistics of word occurence.

First we count the co-occurence of word-word matrix X, $$ X_{ij}$$ be the number of times word j occurs in the context of word i. $$ X_i = \sum_k X_{ik} $$ be the number of times word appears in the context of word i. $$ P_{ji} = P(j\mid i) = \frac{X_{ij}}{X_i} $$ be the probability of word j appearing in the context of word i. Now meaning can be implied from co-occurance probability. Take i = ice and j = steam, in thermodynamic phase context. We can study the relationship of these words by the ratio of their co-occurence probabilities using prob words k.  For k = solid, that is related to ice but not steam, the ratio $$ \frac{P_{ik}}{P_{jk}} $$ will be large. For k = gas that is related to steam but not ice, the ratio is small. For k = water that is not related, the ratio is close to 1. The author also proposed a weighted least squares regression to address the drawback that the model weighs all co-occurrences equally, even the ones that never happen or rarely happen.

They train the model on corpora of Wikipedia (6B tokens) and CommonCrawl (42B tokens). They achieve superior resutls due to some factors: the choice to use negative sampling (which works better than the hierarchical softmax), the number of negative samples, and the choice of the corpus.


<img width="712" alt="Screen Shot 2023-04-07 at 18 54 48" src="https://user-images.githubusercontent.com/7457301/230604671-8d016cd5-0a58-4bf1-8445-4aca26d2b153.png">


## Term frequency - inverse document frequency TFIDF

TF-IDF stands for Term Frequency-Inverse Document Frequency. It is a numerical statistic that is commonly used to evaluate the importance of words in a document or corpus.

The basic idea behind TF-IDF is that words that occur frequently in a document, but rarely in other documents, are more important and informative for that document's content. Conversely, words that occur frequently in all documents (such as the, and) are less important and may be considered stopwords.

TF-IDF is calculated by multiplying two values: term frequency (TF) and inverse document frequency (IDF). The term frequency is simply the number of times a word appears in a document f(w) divided by the total word of that document f(d), while the inverse document frequency is log of total number of documents in the corpus $$ \mid D \mid $$ divided by the number of documents in which the term appears. For the TF, sometimes they use log scaled frequency, sometimes only the raw count. For the IDF, sometimes they add 1 into the nominator, denominator and the term, to avoid divided by zero error.

$$ tfidf = \frac{f(w)}{f(d)} . log \frac{\mid D \mid}{\{\mid d \in D: w \in d \}\mid} $$

After having the TF-IDF we can measure word similarity by cosine similarity (the vector version of the Euclidean distance). Or after having the rank of word importance, we can retrieve documents based on those search term. Another application is after having the most important words, we summarize the document accordingly.

# Code example

In [4]:
import nltk
from nltk import word_tokenize, sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem.porter import * 
nltk.download('gutenberg') 
nltk.download('punkt') 
nltk.download('stopwords')
import string
import gensim
from gensim.models.phrases import Phraser, Phrases 
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import pandas as pd
from bokeh.io import output_notebook, output_file 
from bokeh.plotting import show, figure 
%matplotlib inline

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/nguyenlinhchi/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nguyenlinhchi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nguyenlinhchi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from nltk.corpus import gutenberg

In [6]:
gberg_sent_tokens = sent_tokenize(gutenberg.raw())

In [7]:
gberg_sent_tokens[0]

'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.'

In [8]:
word_tokenize(gberg_sent_tokens[1])


['She',
 'was',
 'the',
 'youngest',
 'of',
 'the',
 'two',
 'daughters',
 'of',
 'a',
 'most',
 'affectionate',
 ',',
 'indulgent',
 'father',
 ';',
 'and',
 'had',
 ',',
 'in',
 'consequence',
 'of',
 'her',
 'sister',
 "'s",
 'marriage',
 ',',
 'been',
 'mistress',
 'of',
 'his',
 'house',
 'from',
 'a',
 'very',
 'early',
 'period',
 '.']

In [9]:
word_tokenize(gberg_sent_tokens[1])[14]

'father'